In [1]:

## !pip install torchvision


In [2]:

import torch
import numpy as np
import os

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt

from PIL import Image


In [3]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score


In [4]:

import torch.optim as optim 
import torch.nn as nn


In [5]:

learning_rate = 0.003
N_Epochs      = 20
batch_size    = 32


In [6]:

data_path = "data/MNISTdata/"

mnist_train = datasets.MNIST(data_path, train=True, download=True)


In [7]:

mnist_test = datasets.MNIST(data_path, train=False, download=True)


In [8]:

mnist_train_tr = datasets.MNIST(data_path, train=True, download=False, 
                                            transform=transforms.Compose([
                                                transforms.ToTensor()
                                            ]))


In [9]:

mnist_test_tr  = datasets.MNIST(data_path, train=False, download=False, 
                                            transform=transforms.Compose([
                                                transforms.ToTensor()
                                            ]))


In [10]:

mnist_train_tr.data.shape


torch.Size([60000, 28, 28])

In [11]:

batch_size = 32
train_dl   = torch.utils.data.DataLoader(mnist_train_tr, batch_size=batch_size, shuffle=True  ) 


In [12]:

test_dl   = torch.utils.data.DataLoader(mnist_test_tr,  batch_size=10000,      shuffle=False )


In [15]:



def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    f1_measure = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
    print('F1-mesure: %.3f' % f1_measure)
    return f1_measure




In [16]:

class Classifier_CNN(nn.Module):
    
    def __init__(self):
        
        super().__init__()
            
        self.model = nn.Sequential(
                
                ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2), 
 
                ## Convolutional Layer 2
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),   
         
 
                ## feed forward layer 
                nn.Flatten(),
                nn.Linear(800, 512),    ## see how to get 800 below on last cell
                nn.ReLU(),

                nn.Linear(512, 10),
                nn.LogSoftmax(dim=1)
        )
 
 
           
    def forward(self, inputs):
            
        return self.model(inputs)
    


In [17]:

def training_loop( N_Epochs, model, loss_fn, opt  ):
    
    losses_list = []
    
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            
            ## print( xb.shape )   ## check this comes out [N, 1, 28, 28]
            ## yb = torch.squeeze(yb, dim=1)
            
            y_pred = model(xb)
            loss   = loss_fn(y_pred, yb)
       
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 1 == 0:
            print(epoch, "loss=", loss)
            losses_list.append(  loss  )
            
    return losses_list


In [18]:

model          = Classifier_CNN()

opt            = torch.optim.Adam(    model.parameters(), lr=learning_rate )

loss_fn        = nn.CrossEntropyLoss( )   

my_losses_list = training_loop(  N_Epochs, model, loss_fn, opt  )


0 loss= tensor(0.0032, grad_fn=<NllLossBackward0>)
1 loss= tensor(0.0703, grad_fn=<NllLossBackward0>)
2 loss= tensor(0.0038, grad_fn=<NllLossBackward0>)
3 loss= tensor(0.0410, grad_fn=<NllLossBackward0>)
4 loss= tensor(0.0009, grad_fn=<NllLossBackward0>)
5 loss= tensor(0.0079, grad_fn=<NllLossBackward0>)
6 loss= tensor(0.0003, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 


## More Examples


In [19]:

N_batches = 1


In [20]:

my_tensor_test = torch.randn(  N_batches , 1, 28,  28)

my_tensor_test.shape


torch.Size([1, 1, 28, 28])

In [21]:

maths_conv1 = nn.Conv2d( 1, 16, kernel_size=5, stride=2)

res_conv1 = maths_conv1(my_tensor_test)

res_conv1.shape


torch.Size([1, 16, 12, 12])

In [22]:

maths_conv2 = nn.Conv2d( 16, 32, kernel_size=5, stride=2)
res_conv2 = maths_conv2(res_conv1)
res_conv2.shape


torch.Size([1, 32, 4, 4])

In [23]:

reshape_res = res_conv2.view(   (N_batches , -1)  )
reshape_res.shape


torch.Size([1, 512])

In [24]:

large_maths_model =  nn.Sequential(
            
              ## Convolution layer 1
              nn.Conv2d(3, 16, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(16),
              nn.MaxPool2d(2, 2),
              nn.Dropout(0.25),
      
              ## Convolution layer2
              nn.Conv2d(16, 32, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(32),
              nn.MaxPool2d(2, 2),
              nn.Dropout(0.25),
        
        )


In [26]:

N_batches = 10


In [27]:

large_my_tensor_test   = torch.randn(N_batches, 3, 256,  256)

large_my_tensor_test.shape


torch.Size([10, 3, 256, 256])

In [28]:

large_res_actual_model = large_maths_model(  large_my_tensor_test   )

large_res_actual_model.shape


torch.Size([10, 32, 15, 15])

In [29]:


large_res_actual_model.view(   (N_batches, -1)  ).shape



torch.Size([10, 7200])

In [30]:

maths_model =  nn.Sequential(
            
              ## Convolution layer 1
              nn.Conv2d(1, 16, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(16),
              nn.Dropout(0.25),
      
              ## Convolution layer2
              nn.Conv2d(16, 32, kernel_size=5, stride=2),
              nn.LeakyReLU(0.2),
              nn.BatchNorm2d(32),
              nn.Dropout(0.25),
        
        )


In [31]:

N_batches = 10


In [32]:

my_tensor_test   = torch.randn(N_batches, 1, 28,  28)

res_actual_model = maths_model(  my_tensor_test   )

res_actual_model.shape


torch.Size([10, 32, 4, 4])

In [33]:

res_actual_model.view(   (N_batches, -1)  ).shape


torch.Size([10, 512])


## Find the 800 value 


In [34]:



model_rc = nn.Sequential(
                    
              
                
                ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),     
 
                ## Convolutional Layer
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(2, 2),     
    
                nn.Flatten()
)




In [35]:



for xb, yb in train_dl:
    
    print( xb.shape )         ## Dataloader converted from [32, 28, 28] to [32, 1, 28, 28]
    break




torch.Size([32, 1, 28, 28])


In [36]:

my_tensor_test   = torch.randn(N_batches, 1, 28,  28)

res_actual_model = model_rc(  my_tensor_test   )

res_actual_model.shape


torch.Size([10, 800])